In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import re

#In order to ignore false-positive chained assignment warning
pd.options.mode.chained_assignment = None

In [2]:
book_dtf = pd.read_csv("C:/Users/joann/Downloads/newProject/Data_Cleaning/Cleaned_Data.csv")

In [3]:
#Deleting the previous indexing
del book_dtf["Unnamed: 0"]

book_dtf.head()

,User_ID,Book_ID,Stars_Ratings,Review,Date_Added,Likes_Count,Num_Review,Num_Ratings,Average_Ratings,Author_Name,Combined_Likes_Count
0,e88c751829dc59cef8e3f23bf10e0f13,1,5,"""I am the Half-blood prince"". \n Read to find ...",Sat Nov 26 10:52:38 -0800 2016,1,22680.0,1713866.0,4.54,"[{'author_id': '1077326', 'role': ''}, {'autho...",7439
1,bd12db2c6bb0e2e26e2b1868a1463b68,1,5,The insights into the life of Tom Riddle do no...,Fri Aug 30 18:56:16 -0700 2013,1,22680.0,1713866.0,4.54,"[{'author_id': '1077326', 'role': ''}, {'autho...",7439
2,5c0a76a445829482c8bbc8243734795a,1,3,"In no way do I diss Harry Potter, but I must s...",Sun Apr 19 16:46:01 -0700 2009,2,22680.0,1713866.0,4.54,"[{'author_id': '1077326', 'role': ''}, {'autho...",7439
3,c96c64adc01b167628026634543278e9,1,5,This was my favorite HP book so far! What a ch...,Tue Jan 06 15:26:36 -0800 2015,1,22680.0,1713866.0,4.54,"[{'author_id': '1077326', 'role': ''}, {'autho...",7439
4,4af2b6a2ed4e32ee9f12f7240c3b6df8,1,5,One of the most moving of the series. It is in...,Thu Jun 23 08:09:30 -0700 2016,1,22680.0,1713866.0,4.54,"[{'author_id': '1077326', 'role': ''}, {'autho...",7439


In [4]:
book_dtf.shape

(2029087, 11)

In [5]:
#Count the number of review words in each review and store it in a new column called Review_Word_Count
list_number = []
for i in range(book_dtf.shape[0]):
    list_number.append(len(re.findall(r'\w+', book_dtf["Review"][i])))

print(len(list_number))

book_dtf["Review_Word_Count"] = list_number

2029087


In [6]:
#To determine if a review is popular or not, we will use a variable called Ratio
    #Ratio = Combined Likes and Comments on the review / Total number of combined likes and reviews of the book
    #Ratio will tell us the relative popularity of the review, and allows a fair comparison between reviews on different books with varying number of readers
    
book_dtf["Ratio"] = book_dtf["Likes_Count"]/book_dtf["Combined_Likes_Count"]
print(book_dtf.shape)

(2029087, 13)


In [7]:
#Books with Ratio less than 0.02 will be considered as unpopular, while those above 0.02 will be considered as popular
#We will label unpopular with 0 and popular with 1

book_dtf["Popularity"] = np.where(book_dtf["Ratio"]> 0.02 ,1,0)
print(book_dtf.shape)

(2029087, 14)


In [8]:
book_dtf["Popularity"].value_counts()

0    1659872
1     369215
Name: Popularity, dtype: int64

In [9]:
# Calculate Number of Reviews made by each user
User_Reviews = book_dtf.groupby("User_ID")["Book_ID"].count().rename("User_Reviews")
print(User_Reviews)

User_ID
0000c3d51aa099745e93a4e99c4856c8    57
0001085188e302fc6b2568de45a5f56b     2
000192962b87d560f00b06fdcbd71681     4
000253cf1077f203bca050637f2ec7f7     9
000259bcbae87b5fd301a6f507ac3f49     1
                                    ..
ffff483c0ce66e6475445eab566bb48f     6
ffff601c0ffa34bd5ffbbf2caee30644    15
ffff7cafdaf5196383cb2efca08fb6fe    98
ffffbb062a8b208c9c1031b529c08f7a     2
fffff8a718843c0e11dfd93fb41c1297     3
Name: User_Reviews, Length: 130539, dtype: int64


In [10]:
book_dtf = book_dtf.merge(User_Reviews,on='User_ID')

In [11]:
#Keeping only the columns that we need
book_dtf = book_dtf[["Book_ID", "User_Reviews", "Stars_Ratings", "Average_Ratings", "Num_Ratings", "Review", "Combined_Likes_Count", "Likes_Count", "Ratio", "Popularity"]]

In [12]:
book_dtf.head()

,Book_ID,User_Reviews,Stars_Ratings,Average_Ratings,Num_Ratings,Review,Combined_Likes_Count,Likes_Count,Ratio,Popularity
0,1,18,5,4.54,1713866.0,"""I am the Half-blood prince"". \n Read to find ...",7439,1,0.000134,0
1,2,18,5,4.47,1766895.0,** spoiler alert ** \n A little memorial for m...,23232,1,0.000043,0
2,6,18,5,4.53,1792561.0,One of my fav books!!,11254,1,0.000089,0
3,24770,18,5,3.86,457310.0,"** spoiler alert ** \n Oh my goodness, it was ...",4130,7,0.001695,0
4,47281,18,4,4.11,338446.0,I thought this was a great and educational boo...,1449,2,0.001380,0


In [13]:
book_dtf.shape

(2029087, 10)

In [14]:
book_dtf.to_csv('Cleaned_Data.csv', encoding='utf-8')